In [119]:
import numpy as np
from math import pi, sqrt, exp
import matplotlib.pyplot as plt
from scipy import signal
from scipy.stats import moyal

from matplotlib.widgets import Slider, Button
%matplotlib widget


def squarewave(time_array, frequency, phase):
    squarewave_temp = signal.square(frequency*time_array + phase)
    squarewave_array = squarewave_mover(squarewave_temp)
    return squarewave_array

def landau(time_array):
    return ((1/sqrt(2*pi)) * exp(-1 * (time_array  + exp(-1*time_array))/2) *4)

def squarewave_mover(squarewave):
    for i in range(len(squarewave)):
        if squarewave[i] < 0:
            squarewave[i] = 0
        else:
            continue
    return squarewave
def landau_array_creator(time_array, landau_x_shift):
    landau_array = np.array([])
    for i in range(len(time_array)):
        landau_array = np.append(landau_array, landau(time_array[i] -landau_x_shift))
    return landau_array

time_array = np.linspace(0, 50, 500)


frequency_timer = .5
frequency_tdc = frequency_timer * 8
phase_timer = 0
phase_tdc = 0
landau_x_shift = 5

squarewave_timer = squarewave(time_array,frequency_timer, phase_timer)
squarewave_tdc = squarewave(time_array,frequency_tdc,phase_timer)


landau_array = landau_array_creator(time_array, landau_x_shift)


# Create the figure and subplots
fig, axs = plt.subplots(3, 1, figsize=(7, 7))
plt.suptitle("Trigger and Distrubution")

axs[0].plot(time_array, landau_array, label = "Landau Distribution")
axs[1].plot(time_array, squarewave_timer, 'tab:red', label = "Timer")
axs[2].plot(time_array, squarewave_tdc, 'tab:green', label = "TDC")
fig.legend()

fig.subplots_adjust(left=0.25, bottom=0.20)


axfreq = fig.add_axes([0.25, 0.05, 0.65, 0.03])
freq_slider = Slider(
    ax=axfreq,
    label='Frequency [Hz]',
    valmin=0.1,
    valmax=2,
    valinit=frequency_timer,
)

axpos = fig.add_axes([0.25, 0.01, 0.65, 0.03])
pos_slider = Slider(
    ax=axpos,
    label='Position of Distrubution',
    valmin=0.1,
    valmax=50,
    valinit=landau_x_shift,
)

# The function to be called anytime a slider's value changes
def update(val):
    phase = 0
    freq_1 = freq_slider.val
    freq_2 = freq_1 * 8
    
    y1 = squarewave(time_array,freq_1,phase)
    y2 =squarewave(time_array,freq_2,phase)
    axs[1].lines[0].set_ydata(y1)
    axs[2].lines[0].set_ydata(y2)
    fig.canvas.draw_idle()

def update2(val):
    x_shift = pos_slider.val
    
    y1 = landau_array_creator(time_array,x_shift)
    axs[0].lines[0].set_ydata(y1)
    fig.canvas.draw_idle()
    
    
freq_slider.on_changed(update)
pos_slider.on_changed(update2)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = fig.add_axes([0.8, 0.1, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')


def reset(event):
    freq_slider.reset()
    pos_slider.reset()
button.on_clicked(reset)


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …